<img width="8%" alt="OWID.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/OWID.png" style="border-radius: 15%">

# OWID - Visualize wealth distribuition of certain major economic powers
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=OWID+-+Visualize+wealth+distribuition+of+certain+major+economic+powers:+Error+short+description">Bug report</a>

**Tags:** #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Last update:** 2023-07-31 (Created: 2023-07-31)

**Description:** This notebook creates an interactive plot using Dash app infrastructure with OWID's wealth distribution data. It shows the percentage of personal wealth obtained by the top 1% and the top 10%.

**References:**
- https://github.com/owid/owid-datasets/tree/master/datasets/Top%20Net%20Personal%20Wealth%20Shares%20%E2%80%93%20WID%20(2018)
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

### Import libraries

In [ ]:
try:
    import dash
    import os
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

### Setup variables
- `DASH_PORT`: specify a port number for Dash
- `url`: URL to get data from Excel
- `title`: App title

In [ ]:
DASH_PORT = 8050
url = "https://raw.githubusercontent.com/owid/owid-datasets/master/datasets/Top%20Net%20Personal%20Wealth%20Shares%20%E2%80%93%20WID%20(2018)/Top%20Net%20Personal%20Wealth%20Shares%20%E2%80%93%20WID%20(2018).csv"
title = "Wealth distribution"

## Model

### Initialize Dash app
The `os.environ.get("JUPYTERHUB_USER")` is used to access the environment variable `JUPYTERHUB_USER` already stored into your Naas Lab.

In [ ]:
app = dash.Dash(
    title=title,
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

#app = dash.Dash(title = "Wealth distribution") if you are not in Naas

### Get Data

In [ ]:
contents_df = pd.read_csv(url, header = 0) 
entity = contents_df["Entity"]
year = contents_df["Year"]
one_pct = contents_df["Top 1% share of net personal wealth (WID 2018)"]# sort the data
ten_pct = contents_df["Top 10% share of net personal wealth (WID 2018)"]

### Create new dataframe

In [ ]:
categories = []
categories1 = []
for i in range(len(entity)):
    categories.append("Top 1%")
    categories1.append("Top 10%")
new_dict = {"Country": entity,
           "Year": year,
            "Categories": categories,
           "Wealth": one_pct}
new_df = pd.DataFrame(data=new_dict)
new_dict1 = {"Country": entity,
           "Year": year,
            "Categories": categories1,
           "Wealth": ten_pct}
new_df1 = pd.DataFrame(data=new_dict1)
new_df = pd.concat([new_df, new_df1])

### Create Dash App

In [ ]:
app.layout = html.Div(
    [
        html.H4("Wealth of top 1% vs wealth of top 10% as a percentage of total wealth for 5 countries"),
        html.P("Select country"),
        dcc.Dropdown(
            id="country",
            options=new_df.Country.unique(),
            value="China"
        ),
        dcc.RangeSlider(id='slider', min=1807, max=2015, value=[1978, 2015],
               marks={x: str(x) for x in [1807, 1857, 1907, 1957, 2007, 2015]}),
        dcc.Graph(id="Wealth", figure={}, style={'display': 'none'})
    ]
)

@callback(
    Output("Wealth", 'figure'),
    Output("Wealth", 'style'),
    Input('country', 'value'),
    Input('slider', 'value')
)

def update_graph(country, year):
    country_list = [country]
    dff = (new_df["Country"].isin(country_list)) & (new_df["Year"] <= year[1]) & (new_df["Year"] >= year[0]) & (new_df["Year"] <= 2021)
        
    figures = px.line(new_df[dff], x='Year', y="Wealth", color = "Categories", markers = True).update_layout(
                                plot_bgcolor='rgba(0, 0, 0, 0)', height= 600)
    styles = {'display': 'block'}   
    return figures, styles

## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")